# $$ \text{ASTR-}119\text{ Homework } 4 $$

In [ ]:
import numpy as np

### $$ f(x) = \int_{a}^{b} e^{-2x}cos(10x)$$

In [ ]:
def func(x):
    return np.exp(-2*x) * np.cos(10*x)

In [ ]:
def func_integral(x):
    return -1/52 * np.exp(-2*x) * (np.cos(10*x) - 5*np.sin(10*x))

## Romberg Integration Method

In [ ]:
def romberg_core(f,a,b,I,i):
    
    # I is an array of integrations
    # i is the iteration that we are on
    # f is function
    # a and b are our limits of integration
    # we need the difference between a and b
    
    h = b-a
    
    #and the increment between new function evals
    # essentially the midpoint, raised to the index we are on
    dh = h/2.**(i)
    
    # we need our cofactor
    K = h/2.**(i+1)
    
    #and the function evaluations
    M = 0.0
    for j in range(2**i):
        M += f(a + 0.5*dh+ j*dh)
        
    #return the answer
    return K*M

In [ ]:
def romberg_integration(f,a,b,tol):
    
    #define an iteration variable
    i = 0
    
    #define a maximum number of iterations
    imax = 1000
    
    #define an error estimate, set to a large value
    delta = 100.0*np.fabs(tol)
    
    #set an array of integral answers
    I = np.zeros(imax, dtype=float)
    
    #get the zeroth romberg iteration
    I[0] = 0.5*(b-a)*(f(a) + f(b))
    
    #iterate by 1
    i += 1
    
    while(delta>tol):
        
        #find this romberg iteration
        I[i] = 0.5*I[i-1] + romberg_core(f,a,b,I,i)
        
        #compute the new fractional error estimate
        delta = np.fabs((I[i]-I[i-1])/I[i])
        
        print(i, I[i], I[i-1], delta)
        
        if(delta>tol):
            
            #iterate
            i+=1
            
            #if we've reached the maximum iterations
            if(i>imax):
                print('Max iterations reached.')
                raise StopIteration('Stopping iterations after', i)
                
    #return the answer
    return I[i]

### It takes 23 iterations using the Romberg method, to reach an accuracy within 1.0e-6.
#### Next, we try to achieve a similar accuracy using the trapezoid and Simpson's methods

## Integration using trapezoid method

In [ ]:
def trapezoid_core(f, x, h):
    return 0.5 * h * ( f(x+h) + f(x) )

In [ ]:
def trapezoid_method(f,a,b,N):
    
    # f == function to integrate
    # a == lower limit of integration
    # b == upper limit of integration
    # N == number of function evaluations to use
    # note the number of chunks will be N-1
    # so if N is odd, then we don't need to adjust the
    # last segment
    
    # define x values to perform trapezoid method
    x = np.linspace(a,b,N)
    h = x[1]-x[0]
    
    #define the value of the integral
    Fint = 0.0
    
    #perform the integral using the trapezoid method
    for i in range(0, len(x)-1, 1):
        Fint += trapezoid_core(f,x[i],h)
        
    return Fint

## Simpson's Method

In [ ]:
def simpson_core(f,x,h):
    return h*(f(x) + 4*f(x+h) + f(x+2*h))/3.

In [ ]:
def simpsons_method(f, a, b, N):
    
    # f == function to integrate
    # a == lower limit of integration
    # b == upper limit of integration
    # N == number of function evaluations to use
    # note the number of chunks will be N-1
    # so if N is odd, then we don't need to adjust the
    # last segment
    
    # define x values to perform trapezoid method
    x = np.linspace(a,b,N)
    h = x[1]-x[0]
    
    #define the value of the integral
    Fint = 0.0
    
    #if N is odd or even, we have different
    #numbers of chunks to add
    if((N%2)==0):
        lim = 3
    else:
        lim = 2
    
    #perform the integral using the simpson's method
    for i in range(0, len(x)-2, 2):
        Fint += simpson_core(f, x[i],h)
        #print(i, i+1, x[i], x[i]+h, x[-2], x[-1])
        
    #apply simpson's rule over the last interval
    #if N is even
    
    #if we have an even number of cores, we split the last 
    #core and treat it as two chunks to fit a parabola to
    if((N%2)==0):
        Fint += simpson_core(f, x[-2], 0.5*h)
        
    #return the answer
    return Fint

## Outputs:

In [ ]:
Answer = func_integral(np.pi)-func_integral(0)
print('Answer:')
print(Answer)

print('Romberg')
tolerance = 1.0e-6
RI = romberg_integration(func, 0, np.pi, tolerance)
print(RI, (RI-Answer)/Answer, tolerance)

print('Trapezoid')
print(trapezoid_method(func, 0, np.pi, 2960))
print('Tolerance of 1.0e-6, within 2960 intervals')
print('Simpson\'s Method')
print(simpsons_method(func,0,np.pi,205))
print('Tolerance of 1.0e-6, within 205 intervals')